In [1]:
!wget https://huggingface.co/CarperAI/stable-vicuna-13b-delta/raw/main/apply_delta.py

--2023-05-28 18:37:57--  https://huggingface.co/CarperAI/stable-vicuna-13b-delta/raw/main/apply_delta.py
Resolving huggingface.co (huggingface.co)... 18.172.170.36, 18.172.170.14, 18.172.170.70, ...
Connecting to huggingface.co (huggingface.co)|18.172.170.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877 (1.8K) [text/plain]
Saving to: ‘apply_delta.py’

apply_delta.py      100%[===================>]   1.83K  --.-KB/s    in 0s      

2023-05-28 18:37:58 (140 MB/s) - ‘apply_delta.py’ saved [1877/1877]



In [1]:
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 4.9 MB/s eta 0:00:0000:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0


In [2]:
!python3 apply_delta.py --base huggyllama/llama-13b --target stable-vicuna-13b --delta CarperAI/stable-vicuna-13b-delta

Loading base model
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Loading checkpoint shards: 100%|██████████████████| 3/3 [01:52<00:00, 37.36s/it]
Loading delta
╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /kaggle/working/apply_delta.py:50 in <module>                                │
│                                                                              │
│   47 │                                                                       │
│   48 │   args = parser.parse_args()                                          │
│   49 │                                                                       │
│ ❱ 50 │   apply_delta(args.base_model_path, args.target_model_path, args.delt │
│   51                                                               

In [4]:
apply_delta = '''
"""
Usage:
python3 apply_delta.py --base /path/to/model_weights/llama-13b --target stable-vicuna-13b --delta pvduy/stable-vicuna-13b-delta
"""
import argparse

import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM


def apply_delta(base_model_path, target_model_path, delta_path):
    print("Loading base model")
    base = AutoModelForCausalLM.from_pretrained(
        base_model_path, torch_dtype=torch.float16, low_cpu_mem_usage=True, device_map='auto')

    print("Loading delta")
    delta = AutoModelForCausalLM.from_pretrained(delta_path, torch_dtype=torch.float16, low_cpu_mem_usage=True, device_map='auto')
    delta_tokenizer = AutoTokenizer.from_pretrained(delta_path)

    DEFAULT_PAD_TOKEN = "[PAD]"
    base_tokenizer = AutoTokenizer.from_pretrained(base_model_path, use_fast=False)
    num_new_tokens = base_tokenizer.add_special_tokens(dict(pad_token=DEFAULT_PAD_TOKEN))

    base.resize_token_embeddings(len(base_tokenizer))
    input_embeddings = base.get_input_embeddings().weight.data
    output_embeddings = base.get_output_embeddings().weight.data
    input_embeddings[-num_new_tokens:] = 0
    output_embeddings[-num_new_tokens:] = 0

    print("Applying delta")
    for name, param in tqdm(base.state_dict().items(), desc="Applying delta"):
        assert name in delta.state_dict()
        param.data += delta.state_dict()[name]

    print("Saving target model")
    base.save_pretrained(target_model_path)
    delta_tokenizer.save_pretrained(target_model_path)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--base-model-path", type=str, required=True)
    parser.add_argument("--target-model-path", type=str, required=True)
    parser.add_argument("--delta-path", type=str, required=True)

    args = parser.parse_args()

    apply_delta(args.base_model_path, args.target_model_path, args.delta_path)
'''

with open('apply_delta.py', 'w') as f:
    f.write(apply_delta)

In [5]:
with open('apply_delta.py', 'r') as f:
    content = f.read()
print(content)


"""
Usage:
python3 apply_delta.py --base /path/to/model_weights/llama-13b --target stable-vicuna-13b --delta pvduy/stable-vicuna-13b-delta
"""
import argparse

import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM


def apply_delta(base_model_path, target_model_path, delta_path):
    print("Loading base model")
    base = AutoModelForCausalLM.from_pretrained(
        base_model_path, torch_dtype=torch.float16, low_cpu_mem_usage=True, device_map='auto')

    print("Loading delta")
    delta = AutoModelForCausalLM.from_pretrained(delta_path, torch_dtype=torch.float16, low_cpu_mem_usage=True, device_map='auto')
    delta_tokenizer = AutoTokenizer.from_pretrained(delta_path)

    DEFAULT_PAD_TOKEN = "[PAD]"
    base_tokenizer = AutoTokenizer.from_pretrained(base_model_path, use_fast=False)
    num_new_tokens = base_tokenizer.add_special_tokens(dict(pad_token=DEFAULT_PAD_TOKEN))

    base.resize_token_embeddings(len(base_tokenizer))
    input_embed